In [51]:
import pandas as pd
import pymysql as pymysql

debug = True

start_date = '2016-01-01'
end_date = '2017-01-01'
resample_interval = '5T'

host = "mariadb.mmto.arizona.edu"
database = "measurements"
table = 'bump'
usr = "webuser"
pw = "Clear7Skies!"

conn = pymysql.connect(host=host, port=3306, user=usr, passwd=pw, db=database)

In [52]:
#  This gets the individual e-series thermocouples parameter names
def get_names(connection):
    df = None
    try:
        # Read data
        with connection.cursor() as cursor:
            sql = f"""SELECT ds_name FROM aaa_parameters WHERE ds_name LIKE 'cell_e_series%'"""
            df = pd.read_sql(sql, conn)
    except Exception as e:
        print(f"Error: {e}")
    return df

In [53]:
def get_data(connection, param, database, start_date, end_date):
    df = None
    try:
        # Read data
        with connection.cursor() as cursor:
            sql = f"""SELECT from_unixtime(timestamp/1000) as ts, value as {param} FROM {param} 
                  WHERE timestamp >= UNIX_TIMESTAMP('{start_date}') * 1000
                  AND timestamp < UNIX_TIMESTAMP('{end_date}') * 1000;"""
            # Using a median filter to remove spikes.  Modify as you wish.
            df = pd.read_sql(sql, conn, index_col='ts').resample(resample_interval).median()
    except Exception as e:
        print(f"Error: {e}")
    return df

In [54]:
df_names = get_names(conn)
df_names

,ds_name
0,cell_e_series_backplate_C
1,cell_e_series_backplate_max_C
2,cell_e_series_backplate_min_C
3,cell_e_series_carrier_offset_C
4,cell_e_series_carrier_reference_C
5,cell_e_series_chamber_ambient_C
6,cell_e_series_count
7,cell_e_series_crate_status
8,cell_e_series_diff_C
9,cell_e_series_frontplate_C


In [55]:
drop_list = [
    "cell_e_series_update",
    "cell_e_series_unit_status",
    "cell_e_series_timestamp",
    "cell_e_series_count",
    "cell_e_series_crate_status",
    "cell_e_series_ijb_raw",
    "cell_e_series_interval",
    "cell_e_series_restarted",
    "cell_e_series_server_status"
]
par_list = [
    "cell_e_series_backplate_C",
    "cell_e_series_chamber_ambient_C",
    "cell_e_series_frontplate_C",
    "cell_e_series_in_front_of_primary_C",
    "cell_e_series_lower_plenum_C",
    "cell_e_series_midplate_C",
    "cell_e_series_outside_ambient_C"
]

In [56]:
df_all = None
for (idx, name) in df_names.itertuples():
    if name in par_list:
        if debug:
            print(f"Getting {name} values...")
        df = get_data(conn, name, database, start_date, end_date)
        if df_all is None:
            df_all = df
        else:
            if df is not None:
                df_all = df_all.join(df, how='inner')

Getting cell_e_series_backplate_C values...
Getting cell_e_series_chamber_ambient_C values...
Getting cell_e_series_frontplate_C values...
Getting cell_e_series_in_front_of_primary_C values...
Getting cell_e_series_lower_plenum_C values...
Getting cell_e_series_midplate_C values...
Getting cell_e_series_outside_ambient_C values...


In [57]:
df_all.tail()

,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C
ts,,,,,,,
2016-12-29 02:30:00,8.23170,10.8269,8.29370,10.39480,8.09510,8.21100,7.99700
2016-12-29 02:35:00,8.24415,10.8604,8.30465,10.42585,8.11265,8.21585,8.06665
2016-12-29 02:40:00,8.25930,11.2178,8.35220,10.67290,8.29470,8.23830,8.17280
2016-12-29 02:45:00,8.24595,11.2333,8.34880,10.73460,8.34190,8.23110,8.12380
2016-12-29 02:50:00,8.23860,11.2488,8.34600,10.84270,8.39810,8.22450,8.12650


In [43]:
df_all.shape

(104915, 7)

In [58]:
df_all.to_csv("../halcoll/data/2016_e_series_trimmed.csv")